In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from __future__ import division
from import_file import *
load_libs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# ipath= "C:\Users\Administrator\Desktop\科研风速数据资料\美国时刻风速数据\上海\虹桥\\9705626661750dat.txt"
# For Shanghai
# df.drop(['NCDC', 'Date','I', 'Type', 'Q', 'Q.1'], 1, inplace=True)
# df.rename(columns={'HrMn':'date','Dir': 'dir', 'Spd':'speed'}, inplace=True)

# ipath = './boscombe_down/3466726903106dat.txt'
ipath = './ciampino/6240476818161dat.txt'
# ipath = 'tiree.txt'
# ipath= "marham.txt"
uipath = unicode(ipath , "utf8")
file2 = open(uipath)
df = pd.read_csv(uipath,header=0, skipinitialspace=True)
df.drop(['USAF', 'NCDC', 'I','Type' ,'QCP','Q','Q.1','I.1','Unnamed: 12'], 1,inplace=True)
df.rename(columns={'Date':'date','Dir': 'dir', 'Spd':'speed'}, inplace=True)
df.describe()

In [ ]:
# select year 1945-2014, dir < 999, speed<999
df = df[(df['dir'] < 999) & (df['speed'] < 999)& (df['date'] < 20150000) ]
df.rename(columns={'speed':'speed_mps'}, inplace=True)
df['speed']=df['speed_mps']
df.index= df['date']
df['speed'].plot(legend=True,figsize=(10,4), grid=True)

In [ ]:
# Data Selection
# Marham, date >1973
# Ciampino, date： 1973~2014
if "ciampino" not in ipath: 
    df = df[ (df['date'] > 19730000) & (df['date'] < 20150000)]
    df = df[df['HrMn'] % 100 <= 0.1]
else: 
    # For Roma Ciampino
    df = df[ (df['date'] > 19720000) & (df['date'] < 20150000)]
    df = df[(df['HrMn'] % 100 == 15)|(df['HrMn'] % 100 == 45)|(df['HrMn'] % 100 == 20)|(df['HrMn'] % 100 == 50)]

In [ ]:
# Using speed unit in knot instead of m/s
df['speed']=df['speed_mps']*1.943845
# need more elaboration, some is not near an integer
df['speed'] =  df['speed'].apply(lambda x: int(round(x)))

In [ ]:
df.describe()

In [ ]:
df['incre'] = df.speed.diff(1)
df['incre'].fillna(0)
df['incre_reverse'] = df.speed.diff(-1)
df['incre_reverse'].fillna(0)
print df.sort('speed',ascending=False).head(20)
bins=np.arange(-15, 15 + 1, 1)
df['incre'].hist(bins=bins)

In [ ]:
# Simple Artefacts Handling
df =  df[ (df['incre'] < 20)&(df['incre_reverse'] < 20)]
df = df[df['dir'] % 10 <= 0.1]
df = df[df['speed'] >= 0.1 ]

## After Simple Aretefacts Hnadling

In [ ]:
# Check the max speed
print df.sort('speed',ascending=False).head(20)

In [ ]:
# Ensure data sampled equally
print df['HrMn'].value_counts()
bins=np.arange(min(df.HrMn), max(df.HrMn) + 100, 50)
df['HrMn'].hist(bins=bins, alpha=0.5,figsize=(15, 6))
plt.suptitle('Data Sampling Time Distribution', fontsize = 20)

In [ ]:
bins=np.arange(min(df.speed), max(df.speed) + 1, 1)
df['speed'].hist(bins=bins, alpha=0.5,figsize=(15, 6))
plt.suptitle('Overall Speed Distribution', fontsize = 20)

In [ ]:
bins=np.arange(min(df.dir), max(df.dir) + 10, 5)
df['dir'].hist(bins=bins, alpha=0.3,figsize=(15, 6))
plt.suptitle('Overall Direction Distribution', fontsize = 20)

In [ ]:
df['speed'].plot(legend=True,figsize=(10,4), grid=True)

In [ ]:
# Comparison between year < 2000, and > 2000, looking for: 
# 1. Odd Even Bias
# 2. Time Shift of Wind Speed Distribution
df_temp = df[df['date'] < 20000000]
bins = np.arange(min(df_temp.speed), max(df_temp.speed) + 1, 1)
df_temp['speed'].hist(bins=bins,alpha = 0.5,figsize=(15, 6))

df_temp = df[df['date'] > 20000000]
bins = np.arange(min(df_temp.speed), max(df_temp.speed) + 1, 1)
df_temp['speed'].hist(bins=bins, alpha = 0.5,figsize=(15, 6))

plt.xlabel("Speed")
plt.ylabel("Counts")
plt.suptitle('Comparison between year < 2000, and > 2000', fontsize = 20)

In [ ]:
# Inspect the time shift of speed and degree distribution, and odd-even bias
def check_time_shift(df):
    for start_time in xrange(19750000,20150000,50000):
        end_time = start_time + 50000 
        sub_df = df[ (df['date'] >= start_time) & (df['date'] < end_time)]
        fig = plt.figure()
        bins = np.arange(min(sub_df.speed), max(sub_df.speed) + 1, 1)
        sub_df['speed'].hist(bins=bins)
        fig.suptitle(start_time)

    for start_time in xrange(19750000,20150000,50000):
        end_time = start_time + 50000 
        sub_df = df[ (df['date'] >= start_time) & (df['date'] < end_time)]
        bins=np.arange(min(sub_df.dir), max(sub_df.dir) + 10, 5)
        fig = plt.figure()
        sub_df['dir'].hist(bins=bins, alpha=0.3,figsize=(15, 6))
        fig.suptitle(start_time)
    
# check_time_shift(df)

# Re-select time range after looking at data

In [ ]:
if "ciampino" in ipath: 
    # For Ciampino
    df = df[ (df['date'] >= 20100000) & (df['date'] < 20150000)]
elif "marham" in ipath:
    # For Marham
    # speed still suffers from odd-even bias
    df = df[ (df['date'] >= 20100000) & (df['date'] < 20150000)]
else: 
    # For Tiree, non is stable
    df = df[ (df['date'] > 19720000) & (df['date'] < 20150000)]

In [ ]:
from windrose import WindroseAxes

df.dir_windrose = df.dir
# 90 degree is in east
ax = WindroseAxes.from_ax()
ax.bar(df.dir_windrose, df.speed, normed=True, opening=0.8, edgecolor='white', nsector=36)
ax.set_legend()

In [ ]:
from windrose import WindAxes
ax = WindAxes.from_ax()
bins=np.arange(min(df.speed), max(df.speed) + 1, 1)
ax, params = ax.pdf(df.speed, bins = bins)

In [ ]:
print params
weibull_params = sp.stats.exponweib.fit(df.speed, floc=0, f0=1)
print weibull_params

In [ ]:
sample = df.speed
ecdf = sm.distributions.ECDF(sample)

x = np.linspace(min(sample), max(sample))
y = ecdf(x)
plt.plot(np.log(x), np.log(-np.log(1-y)),'o')

x = np.linspace(min(sample), max(sample))
w_scale = weibull_params[1]
c_shape = weibull_params[3]
qz=np.exp(-(x/c_shape)**w_scale)
plt.plot(np.log(x), np.log(-np.log(qz)),'-')
plt.show()

In [ ]:
# convert from windrose system to polar system
df['dir'] = 90-df.dir

df['dir'] = df['dir'].apply(lambda x: x + 360 if x < 0 else x)

In [ ]:
# # Randomize angle
def randomize_angle(df):
    df['random_angle_incre'] = 0
    df['random_angle_incre'] = df['random_angle_incre'].apply(lambda x: np.random.uniform(-5,5))
    df['dir_ran']=df['dir']+df['random_angle_incre']
    bins=np.arange(min(df.dir_ran), max(df.dir_ran) + 1, 1)

    df['dir_ran']= df['dir_ran'].apply(lambda x: x + 360 if x < 0 else x)
    df['dir_ran']= df['dir_ran'].apply(lambda x: x - 360 if x > 360 else x)
    df['dir'].hist(bins=bins, alpha=0.5,figsize=(15, 6))
    df['dir_ran'].hist(bins=bins, alpha=0.5,figsize=(15, 6))
    return df

In [ ]:
# Randomize the angle degree, to form a better KDE estimation
df = randomize_angle(df)
df['dir']=df['dir_ran']

In [ ]:
alpha = 0
df['phi'] = df['dir'] - alpha
df['u'] = df['speed'] * np.cos(df['phi'] * pi / 180.0)
df['v'] = df['speed'] * np.sin(df['phi'] * pi / 180.0)
df['x']=df.u
df['y']=df.v
fig, ax = plt.subplots(figsize=(8, 8), dpi=80)
ax.set_aspect('equal')
df.plot(kind='scatter', x='x', y='y', alpha=0.35, ax=ax)
print np.mean(df.x), np.mean(df.y)

In [ ]:
# Check if there is any missing dir 
if 'dir_ran' not in df.columns:
    print df['dir'].value_counts()

# Naive OEN Estimation (Single Gaussian)
Optional, only for demonstration purpose

In [ ]:
run_single_OEN = False

In [ ]:
import pylab 
import scipy.stats as stats
def cor_of_alpha(alpha):
    df['phi'] = df['dir'] - alpha
    df['u'] = df['speed'] * np.cos(df['phi'] * pi / 180.0)
    df['v'] = df['speed'] * np.sin(df['phi'] * pi / 180.0)
    cor = sp.stats.pearsonr(df['u'],df['v'])
    return cor[0]

# Try plot alpha - cor
def plot_cor():
    cors = []
    for alpha in np.arange(0,180,10):
        cor = cor_of_alpha(alpha)
        std_u = np.std(df.u)
        std_v = np.std(df.v)
        comb = [alpha, cor, std_u, std_v]
        cors.append(comb)
        print comb

    plt.scatter(zip(*cors)[0], zip(*cors)[1]) # plot rotation-correlation
    plt.title('Rotation angle - Correlation')
    pylab.show()
    
    pylab.plot(zip(*cors)[0], zip(*cors)[2]) # plot rotation-std_u
    pylab.plot(zip(*cors)[0], zip(*cors)[3]) # plot rotation-std_v
    plt.title('Rotation angle - std')
    pylab.show()
    
if run_single_OEN: plot_cor() 

In [ ]:
def rotate_for_lowest_correlation(df):
    # Find the desired alpha for lowest correlation between u,v
    # alpha is the rotation angle for u-v panel
    alpha = sp.optimize.brentq(cor_of_alpha, 0, 90)
    df['phi'] = df['dir'] - alpha
    df['u'] = df['speed'] * np.cos(df['phi'] * pi / 180.0)
    df['v'] = df['speed'] * np.sin(df['phi'] * pi / 180.0)
    cor = sp.stats.pearsonr(df.u,df.v)
    print 'rotation angle and corresponding correlation: \n', alpha, cor
    return df  # return the dataset that has rorated u,v value

if run_single_OEN: df = rotate_for_lowest_correlation(df)

In [ ]:
# Compare u,v with normal distrubution
## P-P plot, vs Q-Q plot, should be q-q
def plot_against_normal_distribution(df):
    mean,std = np.mean(df.u),np.std(df.u)
    df['u_norm']=df['u'].apply(lambda x: (x-mean)/std)
    print 'u mean and std:', mean, std

    mean,std = np.mean(df.v),np.std(df.v)
    df['v_norm']=df['v'].apply(lambda x: (x-mean)/std)
    print 'v mean and std:', mean, std

    stats.probplot(df.u_norm, dist="norm", plot=pylab)
    stats.probplot(df.v_norm, dist="norm", plot=pylab)

    pylab.show()
    
if run_single_OEN: plot_against_normal_distribution(df)

In [ ]:
from scipy import integrate
from math import *

def calculate_the_probability_distribution_of_windspeed(df):
    u,v = np.mean(df.u), np.mean(df.v)
    sigu, sigv = np.std(df.u), np.std(df.v)
    U = sqrt((u)**2 + (v)**2) # the R value of the centre of the ellipse, in polar coords
    phi = atan(u/v) # the angle of the centre of the elliplse, in polar coords

    def f(V,theta):
        return exp(-1/2* (( (V*cos(theta)-U*cos(phi)) /sigu)**2 + ((V*sin(theta)-U*sin(phi)) /sigv)**2))/(2*np.pi*sigu*sigv)*V

    # x is the speed, y is the probalility
    x_vals = np.linspace(0,50.)
    y_vals =[integrate.nquad(f, [[0, x_val],[0, 2*np.pi]]) for x_val in x_vals]
    return x_vals,y_vals

In [ ]:
def plot_comparison(df):
    sample = df.speed
    ecdf = sm.distributions.ECDF(sample)

    # Empirical distribution, plot points
    x = np.linspace(min(sample), max(sample))
    y = ecdf(x)
    plt.plot(np.log(x), np.log(-np.log(1-y)),'o')

    # Weibull distribution
    x = np.linspace(min(sample), max(sample))
    w_scale = weibull_params[1] # The Weibull need previous code fitting 
    c_shape = weibull_params[3]
    qz=np.exp(-(x/c_shape)**w_scale)
    plt.plot(np.log(x), np.log(-np.log(qz)),'-')

    # OEN distribution
    x, y_ = x_vals, y_vals
    y = np.array(zip(*y_)[0])
    plt.plot(np.log(x), np.log(-np.log(1-y)),'-')

    plt.show()
    
if run_single_OEN:
    x_vals,y_vals = calculate_the_probability_distribution_of_windspeed(df)
    plot_comparison(df)

### Single OEN Fitting

In [ ]:
def run_single_OEN_fitting(df):
    plot_cor()
    df = rotate_for_lowest_correlation(df)
    plot_against_normal_distribution(df)
    x_vals,y_vals = calculate_the_probability_distribution_of_windspeed(df)
    plot_comparison(df)

# 0. Create Input, speed_set

In [ ]:
# speed_set = np.array(zip(-df.x, -df.y)) # Adjust the direction to match for Cook's paper
speed_set = np.array(zip(df.x, df.y))
speed_angle_set = np.array(zip(df.speed, df.dir))

# 1. Kernel Density Estimation

In [ ]:
sample = speed_set
from sklearn.neighbors import KernelDensity
kde = KernelDensity(bandwidth=1).fit(sample) # need to consider the bandwidth for different sites

In [ ]:
# Plot jPDF
from mpl_toolkits.mplot3d import Axes3D
from itertools import product

def plot_3d_prob_density(X,Y,Z):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    X, Y = np.meshgrid(X, Y)
    surf = ax.plot_surface(
        X, Y, Z,rstride=1, cstride=1, cmap='jet',
                       linewidth=0, antialiased=False)
    return plt.show()

def plot_2d_prob_density(X,Y,Z):
    fig, ax = plt.subplots(figsize=(8, 8), dpi=80)
    ax.set_aspect('equal')
    # For docs, see `help(plt.contour)`
    return plt.contourf(X, Y, Z, 10, alpha=.75, cmap='jet')  

X = Y = np.arange(-30, 31, 1)
XX,YY=np.meshgrid(X,Y)
coords=np.array((XX.ravel(), YY.ravel())).T  
Z = np.exp(kde.score_samples(coords)).reshape(len(X),len(Y))

plot_3d_prob_density(X,Y,Z)

plot_2d_prob_density(X,Y,Z)

In [ ]:
# Can get the result, but speed is very slow
def f(x,y):
    # exp to turn log probabilty into normal
    return exp(kde.score_samples([x,y]))

# Check for the corretness of the method, should ~1
# integrate.nquad(f, [[-50, 50],[-50, 50]])

# Query for the prob of a certain point
exp(kde.score_samples([0,0]))

# 2. GMM Estimation, using EM algo

In [ ]:
from sklearn import mixture
import matplotlib as mpl
from matplotlib.patches import Ellipse

In [ ]:
sample= speed_set

# fit a Gaussian Mixture Model with two components
clf = mixture.GMM(n_components=3, covariance_type='full')
clf.fit(sample)

print clf.converged_

for i in [0,1,2]:
    print clf.weights_[i], clf.means_[i], clf.covars_[i]

In [ ]:
from numpy import inf
# Check pdf
def f(x,y):
    # exp to turn log probabilty into normal
    return exp(clf.score([[x,y]]))
# integrate.nquad(f, [[-50, 50],[-50, 50]])

In [ ]:
# how to write the for loop?
fig, ax = plt.subplots()
for i, color in enumerate('rgb'):
    # eigenvalues, and eigen vector
    v, w = np.linalg.eigh(clf._get_covars()[i][:2, :2])

    u1 = w[0] / np.linalg.norm(w[0])
    angle = np.arctan2(u1[1], u1[0])
    angle = 180 * angle / np.pi
    # angle, mean, eigenvalues, 
    print angle, clf.means_[i], v
    # half width stands for std value
    ell = mpl.patches.Ellipse(xy=clf.means_[i], width=2*sqrt(v[0]), height=2*sqrt(v[1]), angle = angle, color=color)
    ell.set_alpha(0.5)
    ax.add_patch(ell)
    
# plt.scatter(sample[:, 0], sample[:, 1],  marker='+', alpha=0.5, color = 'y')
ax.set_aspect('equal')
ax.autoscale()
plt.draw() 

In [ ]:
from sklearn import mixture
from matplotlib.colors import LogNorm
import matplotlib as mpl
from matplotlib.patches import Ellipse

In [ ]:
sample= speed_set
# plot sample point
plt.scatter(sample[:, 0], sample[:, 1], marker='+', alpha=0.5)

x = y = np.linspace(-50, 50)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)[0]
Z = Z.reshape(X.shape)

CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=10.0),
                 levels=np.logspace(0, 1, 10))
CB = plt.colorbar(CS, shrink=0.8, extend='both')
plt.xlim((-50,50))
plt.ylim((-50, 50))

In [ ]:
X = Y = np.arange(-30, 31, 1)
XX,YY=np.meshgrid(X,Y)
coords=np.array((XX.ravel(), YY.ravel())).T  
Z = np.exp(clf.score_samples(coords)[0]).reshape(XX.shape)
    
plot_3d_prob_density(X,Y,Z)

plot_2d_prob_density(X,Y,Z)

## GMM, EM VS. KDE

In [ ]:
panel_array = []
for i in np.arange(-30, 31, 1):
    for j in np.arange(-30, 31, 1):
        panel_array.append([i,j])

points = panel_array
gmm_pdf_result = np.exp(clf.score_samples(points)[0])

In [ ]:
kde_result = np.exp(kde.score_samples(points))
kde_result

In [ ]:
test_point = [0,0]
print exp(kde.score_samples(test_point))
print exp(clf.score_samples(test_point)[0])

### RMSE & R Square

In [ ]:
error_array = np.power(gmm_pdf_result - kde_result,2)
rmse = np.sqrt(np.average(error_array))
print rmse

In [ ]:
# R square measure:
# https://en.wikipedia.org/wiki/Coefficient_of_determination
y_mean = np.mean(kde_result)
SS_tot = np.power(kde_result - y_mean,2)
SS_tot_avg = np.average(SS_tot)
print SS_tot_avg

# The original value is result.fun is log of rmse
SS_res_avg = np.average(error_array)
print SS_res_avg

R_square = 1 - SS_res_avg/SS_tot_avg 
print R_square

In [ ]:
# This calculation speed is very slow
def f(V,theta):
    return exp(clf.score([[V*cos(theta),V*sin(theta)]]))*V
# integrate.nquad(f, [[0, inf],[0, 2*np.pi]])

x_vals_OEN3_EM = np.linspace(0,50)
y_vals_OEN3_EM =[integrate.nquad(f, [[0, x_val],[0, 2*np.pi]]) for x_val in x_vals_OEN3_EM]

In [ ]:
sample = df.speed
ecdf = sm.distributions.ECDF(sample)

# Plot points, with empirical distribution
x = np.linspace(min(sample), max(sample))
y = ecdf(x)
plt.plot(np.log(x), np.log(-np.log(1-y)),'o')

# Weibull distribution
x = np.linspace(min(sample), max(sample))
w_scale = weibull_params[1]
c_shape = weibull_params[3]
qz=np.exp(-(x/c_shape)**w_scale)
plt.plot(np.log(x), np.log(-np.log(qz)),'-')

# OEN3 distribution
x, y_ = x_vals_OEN3_EM, y_vals_OEN3_EM
y = np.array(zip(*y_)[0])
plt.plot(np.log(x), np.log(-np.log(1-y)),'-', color = 'y')

plt.show()

In [ ]:
# # Calculate angle distribution
# def f(V,theta):
#     return exp(clf.score([[V*cos(theta),V*sin(theta)]]))*V

# integrate.nquad(f, [[0, inf],[0, 2*np.pi]])

# x_vals = np.linspace(0,2*np.pi, num=36)
# y_vals =[integrate.nquad(f, [[0, inf],[x_val, x_val+np.pi/18]]) for x_val in x_vals]

In [ ]:
# x, y_ = x_vals, y_vals
# y = np.array(zip(*y_)[0])*len(sample)
# bins=np.arange(min(df.dir), max(df.dir) + 1, 1)
# df['dir'].hist(bins=bins, alpha=0.5,figsize=(15, 6))
# plt.plot(x/np.pi*180, y,'-', color='black')
# plt.show()

# 3. Least Square Prob Estimation

In [ ]:
from sklearn.neighbors import KernelDensity
if not kde:
    sample = speed_set
    kde = KernelDensity(bandwidth=1.0).fit(sample) 

In [ ]:
panel_array = []
for i in np.arange(-30, 31, 1):
    for j in np.arange(-30, 31, 1):
        panel_array.append([i,j])

sample = panel_array

In [ ]:
# very slow if the dataset is too large, e.g. 100,000
if not kde_result.all():
    kde_result = np.exp(kde.score_samples(sample))
kde_result

In [ ]:
print len(kde_result)

In [ ]:
gmm_em_result = []
gmm_em_result.extend(clf.weights_[0:3])
for i in [0,1,2]:
    meanx,meany=clf.means_[i].tolist()
    sigx,sigy = np.sqrt(clf.covars_ [i][0,0]), np.sqrt(clf.covars_ [i][1,1])
    rho = clf.covars_ [i][0,1]/(sigx*sigy)
    gmm_em_result.extend([meanx,meany, sigx,sigy,rho])
    print meanx,meany, sigx,sigy,rho
    
print gmm_em_result

In [ ]:
points = sample

from scipy.stats import multivariate_normal

def create_gaussian_2d(meanx,meany,sigx,sigy,rho):
    sigxy = rho*sigx*sigy
    return multivariate_normal(mean=[meanx,meany], cov=[[sigx**2,sigxy],[sigxy,sigy**2]], allow_singular=True)

def square_error(x0, log_mode = False):
    f1,f2,f3,\
    u1,v1,sigu1,sigv1,rho1, \
    u2,v2,sigu2,sigv2,rho2, \
    u3,v3,sigu3,sigv3,rho3 = x0
    # 1. Generate Mixed Gaussian Model  
    g1 = create_gaussian_2d(u1,v1,sigu1,sigv1,rho1)   
    g2 = create_gaussian_2d(u2,v2,sigu2,sigv2,rho2)
    g3 = create_gaussian_2d(u3,v3,sigu3,sigv3,rho3) 
    def mixed_model(points):
        return f1*g1.pdf(points)+f2*g2.pdf(points)+f3*g3.pdf(points)
    # 2. Calculate the sum of square error
    # kde returns log prob, so need to convert it
    if log_mode:
        error_array = np.power(np.log(mixed_model(points)) - np.log(kde_result),2)
        mse = np.average(error_array)
        rmse = np.sqrt(mse)
        result = rmse
    else:
#         error_array = np.power(mixed_model(points) - cook_kde,2)
        error_array = np.power(mixed_model(points) - kde_result,2)
        mse = np.average(error_array)
        rmse = np.sqrt(mse)
        result = np.log(rmse)
    return result
    # The returned number seems too small, how to deal with it?   

cons = [{'type': 'eq', 'fun': lambda x: -x[0] - x[1] - x[2]+ 1}]
bonds = [(0., 0.99),(0., 0.99),(0., 0.99),
            (-30, 30),(-30, 30),(0., 30),(0., 30),(-0.99, 0.99),
            (-30, 30),(-30, 30),(0., 30),(0., 30),(-0.99, 0.99),
            (-30, 30),(-30, 30),(0., 30),(0., 30),(-0.99, 0.99),]

# from GMM,EM 
x0 = gmm_em_result

result = sp.optimize.minimize(
    square_error,
    x0,
    method = 'SLSQP', 
    bounds = bonds,
    constraints=cons,
    tol = 0.000000000001,
    options = {"maxiter": 500})
result

## LRMSE VS. KDE

In [ ]:
# Least Root Mean Square Error result
exp(result.fun)

In [ ]:
# RMSE / Mode
print exp(result.fun)/np.max(kde_result)
print exp(result.fun)/np.mean(kde_result)

In [ ]:
# R square measure:
# https://en.wikipedia.org/wiki/Coefficient_of_determination

y_mean = np.mean(kde_result)
SS_tot = np.power(kde_result - y_mean,2)
SS_tot_avg = np.average(SS_tot)
print SS_tot_avg

# The original value is result.fun is log of rmse
SS_res_avg = np.square(exp(result.fun))
print SS_res_avg

R_square = 1 - SS_res_avg/SS_tot_avg 
print R_square

In [ ]:
# np.mean(cook_kde)/np.mean(kde_result)

In [ ]:
f1,f2, f3,\
u1,v1,sigu1,sigv1,rho1, \
u2,v2,sigu2,sigv2,rho2, \
u3,v3,sigu3,sigv3,rho3 = result.x

g1 = f1,u1,v1,sigu1,sigv1,rho1
g2 = f2,u2,v2,sigu2,sigv2,rho2
g3 = f3,u3,v3,sigu3,sigv3,rho3
gmm = [g1,g2,g3]

In [ ]:
fig, ax = plt.subplots()

for g in gmm:
    xy_mean = np.matrix([g[1],g[2]])
    sigx, sigy, sigxy = g[3],g[4],g[5]*g[3]*g[4]
    cov_matrix = np.matrix([[sigx**2, sigxy], [sigxy, sigy**2]])

    # eigenvalues, and eigen vector
    w, v = np.linalg.eigh(cov_matrix)

    uu = v[0] / np.linalg.norm(v[0])
    angle_arc = -(np.arctan2(uu[0,1], uu[0,0])+np.pi)
    # rorate angle, counter clock wise
    angle = - 180 * angle_arc / np.pi
    
    transform_matrix = np.matrix([[np.cos(angle_arc ), -np.sin(angle_arc )], [np.sin(angle_arc ), np.cos(angle_arc )]])
    xy_mean_in_uv = transform_matrix * xy_mean.T
    
    # print fraction, rotation agnle, u v mean(in standalone panel), std
    print g[0], xy_mean_in_uv, np.sqrt(w), angle, 
    print ''

    ell = mpl.patches.Ellipse(xy=xy_mean.T, width=2*sqrt(w[0]), height=2*sqrt(w[1]), angle = angle)
    ell.set_alpha(0.5)
    ax.add_patch(ell)


ax.autoscale()
ax.set_aspect('equal')
plt.draw() 

In [ ]:
# Plot probability density 
g1 = create_gaussian_2d(u1,v1,sigu1,sigv1,rho1)   
g2 = create_gaussian_2d(u2,v2,sigu2,sigv2,rho2)
g3 = create_gaussian_2d(u3,v3,sigu3,sigv3,rho3) 
def mixed_model_pdf(points):
    return f1*g1.pdf(points)+f2*g2.pdf(points)+f3*g3.pdf(points)

X = Y = np.arange(-30, 31, 1)
XX,YY=np.meshgrid(X,Y)
coords=np.array((XX.ravel(), YY.ravel())).T  
Z = np.exp(mixed_model_pdf(coords)).reshape(XX.shape)
    
plot_3d_prob_density(X,Y,Z)

plot_2d_prob_density(X,Y,Z)

In [ ]:
def f(V,theta):
    return (mixed_model_pdf([[V*cos(theta),V*sin(theta)]]))*V

x_vals_OEN3_LMSE  = np.linspace(0,50)
y_vals_OEN3_LMSE =[integrate.nquad(f, [[0, x_val],[0, 2*np.pi]]) for x_val in x_vals_OEN3_LMSE ]

In [ ]:
def f(V,theta):
    return np.exp(kde.score_samples([[V*cos(theta),V*sin(theta)]]))*V
# integrate.nquad(f, [[0, 50],[0, 2*np.pi]])

In [ ]:
# Plot v-p
sample = df.speed
ecdf = sm.distributions.ECDF(sample)

# Plot points, with empirical distribution
x = np.linspace(min(sample), max(sample))
y = ecdf(x)
plt.plot(np.log(x), np.log(-np.log(1-y)),'o')

# OEN3 distribution, EM
x, y_ = x_vals_OEN3_EM, y_vals_OEN3_EM
y = np.array(zip(*y_)[0])
plt.plot(np.log(x), np.log(-np.log(1-y)),'-', color = 'y')

# OEN3 distribution, LMSE
x, y_ = x_vals_OEN3_LMSE, y_vals_OEN3_LMSE
y = np.array(zip(*y_)[0])
plt.plot(np.log(x), np.log(-np.log(1-y)),'-', color = 'r')

plt.show()

In [ ]:
def f(V,theta):
    return (mixed_model_pdf([[V*cos(theta),V*sin(theta)]]))*V

# Check, should ~1
# integrate.nquad(f, [[0, inf],[0, 2*np.pi]])

In [ ]:
# Calculate Angle Distribution
x_vals = np.linspace(0,2*np.pi, num=36)
y_vals =[integrate.nquad(f, [[0, inf],[x_val, x_val+np.pi/18]]) for x_val in x_vals]

In [ ]:
x, y_ = x_vals, y_vals
# 1. integrate.nquad returns 2 vaule, result, abserr. Need to abstract the first to plot
# 2. * length of data size, to adjust the plot space
y = np.array(zip(*y_)[0])*len(df['dir']) 

bins=np.arange(0, 360 + 1, 10)
df['dir'].hist(bins=bins, alpha=0.5,figsize=(15, 6))

plt.plot(x/np.pi*180, y,'-', color='black')
plt.show()

In [ ]:
# Calculate Speed Distribution
x_vals = np.arange(0, 35, 1)
y_vals =[integrate.nquad(f, [[x_val, x_val+1],[0, 2*np.pi]]) for x_val in x_vals]

x, y_ = x_vals, y_vals
y = np.array(zip(*y_)[0])*len(df.speed)

bins=np.arange(min(df.speed), max(df.speed) + 1, 1)
df['speed'].hist(bins=bins, alpha=0.5,figsize=(15, 6))

plt.plot(x, y,'-', color='black')
plt.show()

In [ ]:
# df['speed'].value_counts().sort_index()
# how to get the value out?

In [ ]:
fff

# KDE From Cook, Ciampino

In [ ]:
# points
# kde results should from [-30, -30], [-30, -29],[-30, -28] to [30,30]
kde_file = 'cook_kde.csv'
temp_matrix = np.loadtxt(open(kde_file,"rb"),delimiter=",",skiprows=1)
temp_matrix2 = temp_matrix.reshape(-1)
cook_kde = temp_matrix2
cook_kde

In [ ]:
error_array = np.power(kde_result - cook_kde,2)
mse = np.average(error_array)
rmse = np.sqrt(mse)
rmse

## OEN3 from Cook

In [ ]:
from scipy.stats import multivariate_normal, norm   

def create_rotate_norm_dist(meanx,meany,sigx,sigy,angle):
    # angle degree, from u,v to x,y, counter-clockwise
    alpha = angle
    alpha_arc = alpha * pi / 180.0
    transform_matrix = np.matrix([[np.cos(alpha_arc), -np.sin(alpha_arc)], [np.sin(alpha_arc), np.cos(alpha_arc)]])
    cov_matrix = np.matrix([[sigx**2, 0], [0, sigy**2]])
    # cov_matrix
    cov_trans = transform_matrix * cov_matrix * transform_matrix.T
    # need to converte by .tolist()[0]
    norm_mean = np.matrix.dot(transform_matrix,np.array([meanx,meany])).tolist()[0]
    norm_rotate = multivariate_normal(mean=norm_mean, cov=cov_trans, allow_singular=True)
    
    return norm_rotate

# need to change the direction of x,y of Cook, to comply with the standard in this program
g1 = create_rotate_norm_dist(0.0565402,3.5586487,5.809,9.237,10.511)
g2 = create_rotate_norm_dist(4.315,-0.407,3.002,4.173,75.193)
g3 = create_rotate_norm_dist(-0.491,-2.777,1.979,2.571,-25.358)

In [ ]:
# format to mean, std, rho
for g in [g1,g2,g3]:
    sigx,sigy = np.sqrt(g.cov[0,0]), np.sqrt(g.cov[1,1])
    rho = g.cov[0,1]/(sigx*sigy)
    print g.mean, sigx,sigy,rho

In [ ]:
# plot Cook's ellipses results.
# Input from Cook's LMSE result, 
# mean_u, mean_v, std_u, std_v, rotation_angle
gc1 = [0.0565402,3.5586487,5.809,9.237,10.511]
gc2 = [4.315,-0.407,3.002,4.173,75.193]
gc3 = [-0.491,-2.777,1.979,2.571,-25.358]

fig, ax = plt.subplots()
for g in [gc1,gc2,gc3]:
    xy_mean_in_uv = np.matrix([g[0],g[1]])
    std_x, std_y = g[2],g[3]
    
    angle = g[4]
    angle_arc = angle * pi / 180.0
    transform_matrix = np.matrix([[np.cos(angle_arc), -np.sin(angle_arc)], [np.sin(angle_arc), np.cos(angle_arc)]])
    xy_mean = transform_matrix * xy_mean_in_uv.T
    
    print xy_mean, g[2],g[3], angle
    
    ell = mpl.patches.Ellipse(xy=xy_mean, width=2*g[2], height=2*g[3], angle = angle)
    ell.set_alpha(0.5)
    ax.add_patch(ell)

ax.autoscale()
ax.set_aspect('equal')
plt.draw() 

In [ ]:
g=g3
# use np.matrix, not np.array
xy_mean = np.matrix([g.mean[0],g.mean[1]])
cov_matrix = g.cov

# eigenvalues, and eigen vector
w, v = np.linalg.eigh(cov_matrix)

uu = v[0] / np.linalg.norm(v[0])
angle_arc = -(np.arctan2(uu[1], uu[0])+np.pi)
# # rorate angle, counter clock wise
angle = - 180 * angle_arc / np.pi

transform_matrix = np.matrix([[np.cos(angle_arc ), -np.sin(angle_arc )], [np.sin(angle_arc ), np.cos(angle_arc )]])
xy_mean_in_uv = transform_matrix * xy_mean.T

# # print fraction, rotation agnle, u v mean(in standalone panel), std
print xy_mean_in_uv, np.sqrt(w),angle

In [ ]:
f1, f2 = 0.415, 0.298
def mixed_model_pdf(points):
    return f1*g1.pdf(points)+f2*g2.pdf(points)+(1-f1-f2)*g3.pdf(points)

X = Y = np.arange(-30, 31, 1)
XX,YY=np.meshgrid(X,Y)
coords=np.array((XX.ravel(), YY.ravel())).T 
Z = mixed_model_pdf(coords).reshape(XX.shape)

plot_3d_prob_density(X,Y,Z)

plot_2d_prob_density(X,Y,Z)

In [ ]:
# points is [-30,30]
error_array = np.power(mixed_model_pdf(points) - cook_kde,2)
mse = np.average(error_array)
rmse = np.sqrt(mse)

In [ ]:
rmse